# Guía paso a paso del notebook

Este documento contiene implementaciones y comparaciones de clasificadores multiclase. A continuación se explica, por secciones (celdas), qué hace cada parte y qué esperar al ejecutar el notebook.

1) Imports
- `numpy`: operaciones y arrays.
- `BaseEstimator`: base para crear estimadores estilo sklearn.
- `LogisticRegression`: clasificador binario usado como base.
- `OneVsRestClassifier`, `OneVsOneClassifier`: wrappers de sklearn para convertir clasificadores binarios en multiclase.
- `combinations`: utilidad para enumerar pares de clases.

Qué revisar: ejecutar esta celda primero para asegurar que todas las librerías están disponibles.

2) Clase `OVAclasificador`
- Propósito: resolver multiclase mediante One-vs-Rest delegando en `OneVsRestClassifier(LogisticRegression(...))`.
- Métodos:
  - `Fit(X, y)`: entrena el wrapper y guarda `self.clf` y `self.clases_` (etiquetas únicas).
  - `Predict(X)`: devuelve etiquetas predichas (usa `self.clf.predict`).
  - `Predict_proba(X)`: devuelve probabilidades por clase (si `predict_proba` disponible) o intenta `decision_function`.
- Estados importantes: `self.clf`, `self.clases_`.

3) Clase `OVOclasificador`
- Propósito: One-vs-One delegando en `OneVsOneClassifier(LogisticRegression(...))`.
- Métodos:
  - `Fit(X, y)`: entrena `OneVsOneClassifier`, guarda `self.clf`, `self.clases_` y construye `self.modelos` mapeando pares de clases a estimadores binarios.
  - `Predict(X)`: delega en `self.clf.predict`.
- Notas: OVO crea K*(K-1)/2 clasificadores; puede ser más costoso para muchas clases.

4) Clase `SoftmaxRegression`
- Propósito: implementación propia de regresión logística multinomial (softmax) con descenso por gradiente.
- Estructura:
  - `self.W`: pesos shape (d+1, K) donde fila 0 = bias.
  - `Fit(X, y)`: construye matriz one-hot, calcula softmax, gradiente de cross-entropy con L2 y actualiza `W` por iteraciones hasta convergencia ó `max_iter`.
  - `Predict(X)`: calcula probabilidades por softmax y devuelve la clase con mayor prob.
- Recomendación: para producción usar `LogisticRegression(multi_class='multinomial')` de sklearn; la implementación aquí es pedagógica.

5) Celda de evaluación
- Flujo:
  1. Cargar dataset Iris (`X, y`).
  2. `train_test_split(..., stratify=y)` para mantener proporciones por clase.
  3. Escalar con `StandardScaler` ajustado en `X_train`.
  4. Instanciar modelos (`ova`, `ovo`, `sm`) y para cada uno:
     - `Fit(X_train_s, y_train)`
     - `Predict(X_test_s)`
     - medir tiempo y calcular `accuracy`, `classification_report` y `confusion_matrix`.
  5. Mostrar resumen con accuracy y tiempo por modelo.

6) Consejos y edge-cases
- Ejecuta las celdas en orden (imports → definiciones → evaluación).
- Llamar `Predict` antes de `Fit` lanzará `RuntimeError` intencional.
- Si un estimador base no soporta `predict_proba`, el código intenta `decision_function` y aplica sigmoide; esas "probabilidades" pueden no estar calibradas.
- Para muchos labels, OVO puede ser costoso; OVA o la solución multinomial suelen ser mejores.

Si quieres, puedo también:
- Añadir docstrings explicativos dentro de cada clase (reemplazando o ampliando los existentes), o
- Ejecutar la celda de evaluación y pegar aquí los resultados (salida de consola). ¿Qué prefieres?

In [ ]:
# Numpy: operaciones numéricas y arrays.
# Uso: X = np.asarray(X) -> asegura tipo numpy.ndarray, shape (n_samples, n_features)
import numpy as np

# BaseEstimator: clase base para implementar estimadores con API similar a sklearn
# Ejemplo: class MiClasificador(BaseEstimator): def Fit(self, X, y): ...
from sklearn.base import BaseEstimator

# LogisticRegression: clasificador binario
# Parámetros principales: C (float, default=1.0), solver (str, ejemplo 'lbfgs'), max_iter (int)
from sklearn.linear_model import LogisticRegression

# Wrappers para convertir clasificadores binarios en multiclase
# OneVsRestClassifier: entrena K clasificadores (uno por clase)
# OneVsOneClassifier: entrena K*(K-1)/2 clasificadores (uno por cada par de clases)
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

# Utilidad para generar pares ordenados de clases
from itertools import combinations

# Nota: si necesitas conteos rápidos puedes importar Counter desde collections
# from collections import Counter


In [ ]:
# One-vs-All (One-vs-Rest) utilizando LogisticRegression como base
class OVAclasificador(BaseEstimator):

    def __init__(self, C=1.0, solver='lbfgs', max_iter=1000):
        # C: float que controla regularización L2 inversa. Ejemplo: C=1.0 (por defecto).
        # solver: optimizador interno de LogisticRegression. Ejemplo: 'lbfgs'.
        # max_iter: iteraciones máximas del solver. Ejemplo: 1000.
        self.C = C
        self.solver = solver
        self.max_iter = max_iter

        # self.clf: después de Fit contendrá la instancia OneVsRestClassifier entrenada.
        # Antes de Fit: None
        # Después de Fit: OneVsRestClassifier con atributos .estimators_ y .classes_
        self.clf = None

        # self.clases_: array numpy 1D con las etiquetas únicas ordenadas.
        # Antes de Fit: None
        # Después de Fit: np.array([0,1,2]) por ejemplo para Iris
        self.clases_ = None

    def Fit(self, X, y):
        # X: array convertible a numpy, shape (n_samples, n_features)
        # y: array convertible a numpy, shape (n_samples,)
        X = np.asarray(X)
        y = np.asarray(y)

        # Crear estimador base
        base = LogisticRegression(C=self.C, solver=self.solver, max_iter=self.max_iter)

        # OneVsRestClassifier encapsula el estimador base creando K clasificadores binarios
        # Cada clasificador distingue una etiqueta vs resto
        self.clf = OneVsRestClassifier(base)

        # Entrenamiento: tras esto self.clf.estimators_ tiene K estimadores
        self.clf.fit(X, y)

        # Guardar el orden de clases; importante para interpretar predict_proba (columnas)
        self.clases_ = getattr(self.clf, 'classes_', None)

        # Retornar self para compatibilidad con estilo sklearn
        return self

    def Predict(self, X):
        # X: shape (m, n_features) donde m=número de muestras a predecir
        X = np.asarray(X)
        if self.clf is None:
            # Evitar uso antes de entrenar
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")

        # Devuelve array de etiquetas shape (m,)
        return self.clf.predict(X)

    def Predict_proba(self, X):
        # Devuelve probabilidades en shape (m, n_classes)
        X = np.asarray(X)
        if self.clf is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")

        # Si el wrapper soporta predict_proba devuelve directamente la matriz
        if hasattr(self.clf, 'predict_proba'):
            # Cada columna corresponde a la clase en self.clases_
            return self.clf.predict_proba(X)

        # Si predict_proba no está disponible, intentar con decision_function
        if hasattr(self.clf, 'decision_function'):
            df = self.clf.decision_function(X)
            # df puede ser shape (m, n_classes) o (m,) para binario; aplicamos sigmoide
            return 1 / (1 + np.exp(-df))

        # Si ninguna opción está disponible, informar
        raise AttributeError("El clasificador base no soporta predict_proba ni decision_function")

# Ejemplos de uso (no ejecutarlos aquí):
# ova = OVAclasificador(C=0.5, solver='liblinear')
# ova.Fit(X_train, y_train)
# preds = ova.Predict(X_test)          # array shape (n_test,)
# proba = ova.Predict_proba(X_test)    # array shape (n_test, n_classes)


In [ ]:
# One-vs-One (OvO) wrapper delegando en sklearn OneVsOneClassifier
class OVOclasificador(BaseEstimator):

    def __init__(self, C=1.0, solver='lbfgs', max_iter=1000):
        # Hiperparámetros para los clasificadores binarios
        self.C = C
        self.solver = solver
        self.max_iter = max_iter

        # self.modelos: diccionario {(clase_a, clase_b): estimator}
        # Se llena en Fit a partir de self.clf.estimators_
        self.modelos = {}

        # self.clases_: np.array de etiquetas únicas; antes None, después np.unique(y)
        self.clases_ = None

        # self.clf: instancia OneVsOneClassifier después de Fit
        self.clf = None

    def Fit(self, X, y):
        # X: (n_samples, n_features), y: (n_samples,)
        X = np.asarray(X)
        y = np.asarray(y)

        # Construir y entrenar OneVsOneClassifier
        base = LogisticRegression(C=self.C, solver=self.solver, max_iter=self.max_iter)
        self.clf = OneVsOneClassifier(base)
        self.clf.fit(X, y)

        # Guardar clases en el orden que sklearn usa internamente
        self.clases_ = getattr(self.clf, 'classes_', None)

        # estimators_ lista: contiene K*(K-1)/2 estimadores
        estimators = getattr(self.clf, 'estimators_', None)
        if estimators is not None and self.clases_ is not None:
            # Asociar cada estimator con el par de clases correspondiente
            # combinations(self.clases_, 2) genera pares en el mismo orden esperado
            for (a, b), est in zip(list(combinations(self.clases_, 2)), estimators):
                self.modelos[(a, b)] = est
        else:
            self.modelos = {}

        return self

    def Predict(self, X):
        # X: (m, n_features)
        X = np.asarray(X)
        if self.clf is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")

        # OneVsOneClassifier realiza la votación internamente y devuelve etiquetas
        return self.clf.predict(X)

# Ejemplo: para K=3 (clases {0,1,2}) se crean 3 clasificadores: (0,1),(0,2),(1,2)
# estimators_ tendrá 3 elementos; self.modelos contendrá claves (0,1),(0,2),(1,2)


In [ ]:
# Implementación de regresión logística multinomial (softmax) con descenso por gradiente
class SoftmaxRegression(BaseEstimator):
    """Softmax implementado manualmente.

    Parámetros y ejemplos:
    - C: (opcional) si se proporciona, reg = 1/C (float). Ejemplo: C=1.0 -> reg=1.0
    - lr: learning rate (float). Recomendado: 0.01 - 1.0 según problema. Valor por defecto 0.1.
    - max_iter: iteraciones máximas (int), ejemplo 1000 o 2000.
    - tol: tolerancia para convergencia en norma de cambio de W, ejemplo 1e-5.
    - reg: regularización L2 directa (si se prefiere pasar explicitamente). Si reg no es None usa ese valor.

    Shapes importantes:
    - X: (n, d)
    - Xb (con bias): (n, d+1)
    - W: (d+1, K) donde K = número de clases
    - probs: (n, K)
    """

    def __init__(self, C=None, lr=0.1, max_iter=1000, tol=1e-5, reg=None, verbose=False, **kwargs):
        # Determinar regularización: si reg no se pasa, tomar 1/C si C fue provisto
        if reg is None:
            if C is None:
                # Valor por defecto pequeño para evitar overfitting sin C
                self.reg = 1e-3
            else:
                self.reg = 1.0 / C
        else:
            self.reg = reg

        # Learning rate: controla tamaño de paso en descenso por gradiente
        self.lr = lr
        # Iteraciones máximas
        self.max_iter = max_iter
        # Tolerancia de convergencia en norma de cambio de W
        self.tol = tol
        # Verbose para imprimir progreso
        self.verbose = verbose

        # Pesos W inicializados en Fit; antes None
        self.W = None
        # Clases detectadas en Fit; antes None
        self.classes_ = None

    def _one_hot(self, y_idx, K):
        # y_idx: vector de índices en rango 0..K-1, shape (n,)
        n = y_idx.shape[0]
        Y = np.zeros((n, K), dtype=float)
        Y[np.arange(n), y_idx] = 1.0
        return Y

    def Fit(self, X, y):
        # Asegurar arrays numpy
        X = np.asarray(X)
        y = np.asarray(y)

        # Detectar clases únicas y mapear a índices 0..K-1
        self.classes_ = np.unique(y)
        K = len(self.classes_)
        class_to_idx = {c: i for i, c in enumerate(self.classes_)}
        # y_idx contiene índices 0..K-1 correspondientemente
        y_idx = np.vectorize(class_to_idx.get)(y)

        # Añadir columna de unos para bias: Xb shape (n, d+1)
        n, d = X.shape
        Xb = np.hstack([np.ones((n, 1)), X])

        # Inicializar W: ceros (d+1, K). Alternativa: pequeña aleatoriedad ayuda a romper simetrías
        self.W = np.zeros((d + 1, K), dtype=float)

        # Matriz one-hot de etiquetas Y shape (n, K)
        Y = self._one_hot(y_idx, K)

        # Descenso por gradiente
        for it in range(self.max_iter):
            # logits: Xb.dot(W) -> shape (n, K)
            scores = Xb.dot(self.W)
            # Estabilizar antes de softmax
            scores -= scores.max(axis=1, keepdims=True)
            exp_scores = np.exp(scores)
            probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)  # (n, K)

            # Gradiente de la función de pérdida (cross-entropy + L2)
            grad = - (Xb.T.dot(Y - probs)) / n  # shape (d+1, K)
            # Regularización L2 aplicada solo a pesos (no bias): primera fila de W no penalizada
            reg_term = self.reg * np.vstack([np.zeros((1, K)), self.W[1:, :]])
            grad += reg_term

            # Actualizar W
            W_old = self.W.copy()
            self.W -= self.lr * grad

            # Norm of change: criterio de parada
            diff = np.linalg.norm(self.W - W_old)
            if self.verbose and (it % 100 == 0 or it == self.max_iter - 1):
                # Loss aproximado: cross-entropy media + L2 (sin bias)
                loss = -np.mean(np.sum(Y * np.log(probs + 1e-15), axis=1)) + 0.5 * self.reg * np.sum(self.W[1:, :] ** 2)
                print(f"it={it} loss={loss:.6f} ||dW||={diff:.6e}")
            if diff < self.tol:
                # Convergencia alcanzada
                break

        return self

    def Predict(self, X):
        # Comprobar que W fue entrenada
        if self.W is None:
            raise RuntimeError("El clasificador no está entrenado. Ejecutar Fit primero.")
        X = np.asarray(X)
        n = X.shape[0]
        Xb = np.hstack([np.ones((n, 1)), X])

        # Calcular probabilidades y devolver etiqueta con mayor probabilidad
        scores = Xb.dot(self.W)
        scores -= scores.max(axis=1, keepdims=True)
        exp_scores = np.exp(scores)
        probs = exp_scores / exp_scores.sum(axis=1, keepdims=True)  # (n, K)
        idx = np.argmax(probs, axis=1)
        return self.classes_[idx]

# Ejemplo de parámetros recomendados:
# - lr=0.1, reg=1e-3 para datasets pequeños.
# - Aumentar max_iter a 2000 si no converge.
# - Usar verbose=True para ver progreso cada 100 it.

In [5]:
# Evaluación comparativa en el dataset Iris con explicaciones detalladas
# Carga del dataset: 'data' contiene X (data.data) y y (data.target)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# data: Bunch con campos .data (X) y .target (y)
data = load_iris()
# X: array (150, 4), y: array (150,) con etiquetas {0,1,2}
X, y = data.data, data.target

# Separar entrenamiento/prueba manteniendo proporciones de clase con 'stratify'
# X_train: (105,4), X_test: (45,4) cuando test_size=0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Escalado: ajustar scaler en X_train y aplicar al conjunto de prueba
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)  # media ~0, desviación ~1 (por columna)
X_test_s = scaler.transform(X_test)

# Instancias de los modelos. Estos objetos estarán sin entrenar hasta llamar a Fit.
ova = OVAclasificador()
ovo = OVOclasificador()
try:
    # intentar con firma de la implementación propia (lr, reg, max_iter)
    sm = SoftmaxRegression(lr=0.5, max_iter=2000, tol=1e-7, verbose=False, reg=1e-3)
except TypeError:
    # fallback por compatibilidad
    sm = SoftmaxRegression(C=1.0, solver='lbfgs', max_iter=2000)

# Lista de tuplas (nombre, objeto) para iterar y comparar
models = [('OvA', ova), ('OvO', ovo), ('Softmax', sm)]

# results: dict donde guardamos info por modelo
# keys: nombre modelo
# values: dict con 'accuracy' (float), 'time_s' (float), 'y_pred' (array predicciones)
results = {}

for name, model in models:
    # t0 guarda el tiempo antes de entrenar y predecir
    t0 = time.time()

    # Entrenar el modelo con X_train_s (forma (n_train, n_features))
    # Después de Fit, cada objeto tendrá internamente coeficientes/estados entrenados
    model.Fit(X_train_s, y_train)

    # Predict sobre X_test_s -> devuelve array (n_test,) con etiquetas predichas
    y_pred = model.Predict(X_test_s)

    # t1 guarda el tiempo después de entrenamiento+predicción
    t1 = time.time()

    # accuracy: número de aciertos / n_test
    acc = accuracy_score(y_test, y_pred)

    # Guardar resultados para posterior inspección
    results[name] = {'accuracy': acc, 'time_s': t1 - t0, 'y_pred': y_pred}

    # Imprimir métricas detalladas para este modelo
    print(f"{name}: accuracy={acc:.4f} time={t1-t0:.3f}s")
    # classification_report muestra precision/recall/f1 por clase (usa labels originales)
    print(classification_report(y_test, y_pred, target_names=data.target_names))
    # confusion_matrix: matriz (n_clases, n_clases) donde rows=true, cols=pred
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("-" * 60)

# Resumen compacto: mostrar accuracy y tiempos guardados en results
print("Resumen:")
for name, info in results.items():
    print(f"{name}: acc={info['accuracy']:.4f} time={info['time_s']:.3f}s")

# Notas de interpretación:
# - Si un modelo tiene baja precisión para una clase, inspeccionar su confusion matrix
# - Softmax (implementación propia) puede requerir ajuste de lr y reg para converger bien
# - Los tiempos reportados incluyen tanto entrenamiento como predicción


OvA: accuracy=0.8444 time=0.050s
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.79      0.73      0.76        15
   virginica       0.75      0.80      0.77        15

    accuracy                           0.84        45
   macro avg       0.85      0.84      0.84        45
weighted avg       0.85      0.84      0.84        45

Confusion matrix:
 [[15  0  0]
 [ 0 11  4]
 [ 0  3 12]]
------------------------------------------------------------
OvO: accuracy=0.9111 time=0.008s
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.82      0.93      0.88        15
   virginica       0.92      0.80      0.86        15

    accuracy                           0.91        45
   macro avg       0.92      0.91      0.91        45
weighted avg       0.92      0.91      0.91        45

Confusion matrix:
 [[15  0  0]
 [ 0 14  1]
 [ 0  3 12]]